In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../../../data'
group_dir_ = os.path.join(dir_, 'groups/low')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

1057

In [3]:
pop_dict = {}
for i in tqdm(df['tid'].unique()):
    pop_dict[i] = len(df[df['tid']==i])/num_user

In [4]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,9,2.243909,1
1,0,15,2.160610,1
2,0,22,2.145235,1
3,0,28,2.204445,1
4,0,33,2.166677,1


In [5]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
70,70,3.637654
5521,5521,2.625355
390,390,2.544939
2519,2519,2.526017
83,83,2.521287
13496,13496,2.511826
210,210,2.421949
5716,5716,2.417219
1036,1036,2.294229
1464,1464,2.280038


In [6]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [7]:
top_n_size = 20

In [8]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*0):int(len(groups)/5*1)]
    groups_n.append(groups)
    
NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.05258790602333653, 0.0418362987151707, 0.031723348105362724, 0.028754821337602775, 0.026126528609972315, 0.02006931438611395, 0.02223371964679911]
[0.07337722064543278, 0.05138859218115781, 0.037186301847333535, 0.0351342697038061, 0.028867514629104033, 0.024894290734269107, 0.02346755755282245]
[0.09768047671607313, 0.06172266813155502, 0.043904670616939645, 0.039727094097954996, 0.03382476610953432, 0.02732801729963509, 0.027184839167455063]
[0.12806751287711593, 0.07513707897172966, 0.0520085395608225, 0.04671930718288329, 0.03939871754441288, 0.029249126329812913, 0.029099653106275616]
[0.15355796015978163, 0.09331797235023066, 0.06271784593935162, 0.05349683429153621, 0.041220785591646485, 0.03172726044060009, 0.029304635761589398]
[0.180248081572585, 0.11057466353343348, 0.07564855848229839, 0.06117022050796876, 0.04738866113038293, 0.0346938775510204, 0.030424156417533894]
[0.18120762377798816, 0.13304442477696135, 0.08910085644575005, 0.0703806127647186, 0.05238182837520583

In [9]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*1):int(len(groups)/5*2)]
    groups_n.append(groups)
    
NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.05041554977399349, 0.041667652160201796, 0.033113205198426504, 0.033010697911360146, 0.028075528224534844, 0.02504645898094337, 0.025288749605802584]
[0.06031516083254504, 0.04756041075370541, 0.03700766817706515, 0.03394585546903425, 0.02955376852727846, 0.02558115961616434, 0.02529466256701356]
[0.07504763218753313, 0.05419327499211598, 0.04146292884529199, 0.03580525434830069, 0.031065121412803525, 0.028813859981078528, 0.029040523494165884]
[0.08858338589298895, 0.06079263245033103, 0.047641039685305926, 0.039139800596754204, 0.03466256701356039, 0.03041711717799703, 0.029721499526963107]
[0.09847642699463931, 0.06766546436455366, 0.05241933476074285, 0.0450149188559784, 0.035551876379690935, 0.03160731179889174, 0.029924511195206564]
[0.10445475927677962, 0.07038345553453161, 0.055645321914056606, 0.05028673313441519, 0.03861084831283504, 0.03520323692390863, 0.034422303689687794]
[0.1043401135288557, 0.07403717281614629, 0.05654844893691176, 0.04955898406229527, 0.04119914853

In [10]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*2):int(len(groups)/5*3)]
    groups_n.append(groups)
    
NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.04809931146851674, 0.03515304714601069, 0.037088457899716154, 0.03416054144530963, 0.025617575948701773, 0.027007872685498036, 0.026252562283191422]
[0.055740171344476064, 0.038557434563229215, 0.038207190160832516, 0.035089877010406785, 0.027263569150986364, 0.02701716448168671, 0.02564451277199621]
[0.06490394723010634, 0.04551846814884888, 0.04352826395458842, 0.03712320791790987, 0.028392725743719118, 0.029695735910258137, 0.026268330179754024]
[0.07328425575528252, 0.051801482182276794, 0.0467029328287606, 0.042451058874899895, 0.03193787448754336, 0.030946749560751458, 0.027779091769157997]
[0.07856748659728828, 0.05629779643645527, 0.052586920529801275, 0.047441234262426246, 0.033769578471565226, 0.03327054331666441, 0.026627049826553135]
[0.08351663513087401, 0.05773957347839787, 0.057771996215704766, 0.046607233825776806, 0.032841024562878865, 0.03422675361535342, 0.028213694418164615]
[0.0826743666561552, 0.06047431803847353, 0.059811967833490944, 0.04674841714576807, 0.0

In [11]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*3):int(len(groups)/5*4)]
    groups_n.append(groups)
    
NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.046582308420056756, 0.045269631031220385, 0.03616304834935016, 0.041015209955607285, 0.0292675812046673, 0.03210484524935801, 0.030068393251340267]
[0.051993324923788525, 0.04591020182907596, 0.03502215804411689, 0.04285350411178223, 0.027307631661936296, 0.032638701175834554, 0.029454430778934088]
[0.05967261904761915, 0.04901549195837269, 0.03779315839267038, 0.043416781893603054, 0.029337748344370862, 0.03432558453845113, 0.029534255755282245]
[0.06556816987280581, 0.05070167139703557, 0.035523826121595346, 0.045001091623608144, 0.03098943550930306, 0.034520712258413286, 0.03156338694418164]
[0.06803157521286678, 0.05254306606748651, 0.03945812378628688, 0.04547631176770245, 0.03294623147272154, 0.03486281929990538, 0.029382489750867227]
[0.06988627404604247, 0.05405826237779871, 0.0381367325598765, 0.045847463794483616, 0.03294623147272154, 0.03298672117853763, 0.028926206244087035]
[0.07032317617996449, 0.055208333333333214, 0.04082370661753722, 0.040064041918346546, 0.0353941

In [12]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups = groups[int(len(groups)/5*4):int(len(groups)/5*5)]
    groups_n.append(groups)
    
NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        
        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    nov += (1-pop_dict[tid])
                    if len(t) > 0:
                        hits[int(tid)] = True
                        hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
# for NOV in NOVs:
#     print(NOV)
# print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
# for COV in COVs:
#     print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.04922758906701574, 0.0441816461684011, 0.039992313150425694, 0.03409041762400322, 0.0391051718700725, 0.03413214623597782, 0.03602344911474522]
[0.054922823706276575, 0.04580081204667293, 0.04297185430463572, 0.0366400864981754, 0.042071901608325406, 0.03361011623192322, 0.0360259832409785]
[0.057501198792135985, 0.049617135761589354, 0.0453837511825922, 0.04179821597513174, 0.046063544623147226, 0.03632501013650491, 0.03660798756588727]
[0.06036663599486795, 0.05108552112898133, 0.045254848628192956, 0.04282470604135689, 0.043862346263008466, 0.03902892282740907, 0.03722040140559533]
[0.06316662562693603, 0.052634224219489045, 0.04719193472090818, 0.043369374239762094, 0.04303768527278457, 0.04079098526827946, 0.036784531693472074]
[0.06615874599862646, 0.052494776884263565, 0.05001951277199618, 0.04590958237599672, 0.04264427625354773, 0.04315785917015808, 0.03964978375456141]
[0.06554736200930542, 0.05285645301166814, 0.04807355723746449, 0.04472158399783752, 0.04143803216650896